<a href="https://colab.research.google.com/github/jdasam/ant5015/blob/2024F/notebooks/6_automatic_music_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Download Dataset

In [ ]:
!gdown --id 1L99FSzloXGQsmc-OwsVKggvQNF9niN2M
!unzip -q maestro_small.zip


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1L99FSzloXGQsmc-OwsVKggvQNF9niN2M
From (redirected): https://drive.google.com/uc?id=1L99FSzloXGQsmc-OwsVKggvQNF9niN2M&confirm=t&uuid=f367aea2-a463-43db-bfbe-678e2616f1bb
To: /content/maestro_small.zip
100% 1.14G/1.14G [00:17<00:00, 64.9MB/s]


In [ ]:
!pip install mido pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 9.6 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=ddaae90c9346a1b949e5265121e9b1c9bddad0660635147e75d9b6b5bd548bc4
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


## 1. Preprocess Data

## 2. Model

In [ ]:
import torch.nn as nn

class SpecConverter(nn.Module):
  def __init__(self):
    super().__init__()
    self.mel = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=2048, hop_length=512, n_mels=352)
    self.db = torchaudio.transforms.AmplitudeToDB()

  def forward(self, x):
    return self.db(self.mel(x)) / 80

converter = SpecConverter()
out = converter(audio)

## 3. Training